In [10]:
import binance
import pandas as pd
from ipywidgets import widgets
import seaborn as sns
import datetime
from matplotlib import pyplot as plt
import time
import datetime
import os
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import train_test_split
import random
import binance
import pandas as pd
from ipywidgets import widgets
import seaborn as sns
import datetime
from matplotlib import pyplot as plt
import time
import datetime
import os
import requests

X_columns = ["gap", "overall_gap", "where_in_gap", "numTrades"] # columns we will keep for 1st preprocessing steps
#X_columns_final = ["gap", "overall_gap", "where_in_gap", "numTrades","buyer_is_maker"]# columns we will keep for last preprocessing steps (after trades have been added)
X_columns_final = ["ma500_trend","numTrades","buyer_is_maker","cons_mvt","tot_mvt","close-ma20","close-ma100", "close-ma500", "close-ma1000", "close-ma2000","ma20-ma100", "ma20-ma500", "ma20-ma1000", "ma20-ma2000" , "ma100-ma500" , "ma100-ma1000" , "ma100-ma2000" , "ma500-ma1000" , "ma500-ma2000" , "ma1000-ma2000", "gap", "ma20_gap", "ma100_gap", "ma500_gap", "ma1000_gap", "ma2000_gap"]
# columns we will keep for last preprocessing steps (after trades have been added)
y_columns = ['is_l100','is_h100'] # our Y columns
#time_step= 100 # how many time units used to explain each output
time_step= 10 # how many time units used to explain each output

check_steps = [10,100,1000,10000,50000,100000,150000] # used as monitoring steps for progress during long calculation processes

start_point = '1544844000000' # timestamp (in ms) to start from: Dec 15th 2018, when we started data collection from
current_point = 1556184840000 # current time: Apr 25th 11:34 am, when I started traininf the LSTM network
test_point = 1559477896000  # June 2nd, when i'm trying to run the algo on real fresh data

low_limit = .0001 # values to ignore in your balance (Not Used yet, will be for further integration with Binance Portofolio)

btcusd = 'BTCUSDT'# the exchange we're using as BTC valuation in USD


# depth of history for min, hrs, day, month when we call Binance APIs
depth_min = '1000'
depth_hrs = '120'
depth_day = '60'
depth_mth = '5'

keys = pd.read_csv("binancekey.csv")# your APIs keys and Secret must be stored in a csv file on root directory.
api_key = keys.loc[0,"api_key"]
api_secret = keys.loc[0,"api_secret"]

binance.set(api_key, api_secret)


### Reusable function for time conversion

# converts time stamp to standard date/time
def clean_date(histo, value):
    histo["closeTime"] = histo["closeTime"].apply(lambda x: datetime.datetime.fromtimestamp(x/1000))
    histo["openTime"] = histo["openTime"].apply(lambda x: datetime.datetime.fromtimestamp(x/1000))
    histo["xch"] = value
    return histo

# collects history of valuations by minutes/day/hours/month. 
def build_histo(minutes, hours, days, months):
    Hminutes = clean_date(pd.DataFrame(binance.klines(btcusd,minutes,limit = depth_min)),btcusd)
    Hhours = clean_date(pd.DataFrame(binance.klines(btcusd,hours,limit = depth_hrs)), btcusd)
    Hdays = clean_date(pd.DataFrame(binance.klines(btcusd,days,limit = depth_day)), btcusd)
    Hmonths = clean_date(pd.DataFrame(binance.klines(btcusd,months,limit = depth_mth)), btcusd)
    return Hminutes, Hhours, Hdays, Hmonths

In [ ]:
# collect Balances
bal = pd.DataFrame(binance.balances()).transpose()["free"].astype(float)
bal = bal[bal > low_limit]

# collect current prices
price = pd.Series(binance.prices())

# build a short history wiht various aggregations:
Hminutes, Hhours, Hdays, Hmonths = build_histo('1m', '1h', '1d', '1M')

# build "minutes" history of klines, and export to files
stop = False
counter = 0
limit_cycles = 100

#initialize:
temp = binance.klines(btcusd,'1m',limit = depth_min, startTime = current_point)
restart = pd.DataFrame(temp)["closeTime"].astype(int).max() + 1
result = temp
temp = []

while stop == False and restart < test_point:
    try: 
        temp = binance.klines(btcusd,'1m',limit = depth_min, startTime = restart)
        restart = pd.DataFrame(temp)["closeTime"].astype(int).max() + 1
        result.extend(temp)
        temp = []
        counter += 1
        print("done {} cycles, {} records, up to {}".format(counter, len(result), datetime.datetime.fromtimestamp(int(restart) /1000)))
        
        if counter == limit_cycles:
            filename = "test/archives stocks/Klines_BTC_USD_" + str(limit_cycles) + ".csv"
            pd.DataFrame(result).to_csv(filename)
            limit_cycles += 100
            result = []
        else:   
            time.sleep(1)
    except:
        stop = True
        print("Unexpected error:", sys.exc_info()[0])
        raise

filename = "test/archives stocks/Klines_BTC_USD_" + str(limit_cycles) + ".csv"
pd.DataFrame(result).to_csv(filename)        
print("job finished :)")  

In [15]:
counter

17102

In [16]:
stop=False

### build trade history from "trades"
'''
stop = False
counter = 0
limit_cycles = 100

#initialize:
temp = requests.get("http://www.binance.com/api/v1/aggTrades", params ={"symbol":btcusd,"startTime" : current_point, "endTime" : int(current_point)+3500000} ,headers={"X-MBX-APIKEY":api_key})

restart = pd.DataFrame(temp.json())["a"].astype(int).max() + 1
result = temp.json()
temp = []
'''

while stop == False and restart < test_point:
    try: 
        temp = requests.get("http://www.binance.com/api/v1/aggTrades", params ={"symbol":btcusd, "fromId" : restart} ,headers={"X-MBX-APIKEY":api_key})
        restart = pd.DataFrame(temp.json())["a"].astype(int).max() + 1
        progress = pd.DataFrame(temp.json())["T"].astype(int).max() + 1
        result.extend(temp.json())
        temp = []
        counter += 1
        print("done {} cycles, {} records, up to {}".format(counter, len(result), datetime.datetime.fromtimestamp(int(progress) /1000)))
        
        if counter == limit_cycles:
            filename = "test/archives trades/Trades_BTCUSD_" + str(limit_cycles) + ".csv"
            pd.DataFrame(result).to_csv(filename)
            limit_cycles += 100
            result = []
        else:   
            time.sleep(1)
    except:
        stop = True
        print("Unexpected error:", sys.exc_info()[0])
        raise

filename = "test/archives trades/Trades_BTCUSD_" + str(limit_cycles) + ".csv"
pd.DataFrame(result).to_csv(filename)        
print("job finished :)")     


done 17103 cycles, 1500 records, up to 2019-05-20 17:43:14.759000
done 17104 cycles, 2000 records, up to 2019-05-20 17:44:56.010000
done 17105 cycles, 2500 records, up to 2019-05-20 17:46:35.676000
done 17106 cycles, 3000 records, up to 2019-05-20 17:48:02.943000
done 17107 cycles, 3500 records, up to 2019-05-20 17:49:48.731000
done 17108 cycles, 4000 records, up to 2019-05-20 17:51:18.867000
done 17109 cycles, 4500 records, up to 2019-05-20 17:53:08.118000
done 17110 cycles, 5000 records, up to 2019-05-20 17:54:48.929000
done 17111 cycles, 5500 records, up to 2019-05-20 17:55:49.488000
done 17112 cycles, 6000 records, up to 2019-05-20 17:56:41.690000
done 17113 cycles, 6500 records, up to 2019-05-20 17:57:38.011000
done 17114 cycles, 7000 records, up to 2019-05-20 17:59:08.623000
done 17115 cycles, 7500 records, up to 2019-05-20 18:00:06.080000
done 17116 cycles, 8000 records, up to 2019-05-20 18:00:48.290000
done 17117 cycles, 8500 records, up to 2019-05-20 18:01:33.058000
done 17118

done 17226 cycles, 13000 records, up to 2019-05-20 20:09:10.895000
done 17227 cycles, 13500 records, up to 2019-05-20 20:10:27.198000
done 17228 cycles, 14000 records, up to 2019-05-20 20:12:01.339000
done 17229 cycles, 14500 records, up to 2019-05-20 20:13:15.981000
done 17230 cycles, 15000 records, up to 2019-05-20 20:15:18.170000
done 17231 cycles, 15500 records, up to 2019-05-20 20:16:58.379000
done 17232 cycles, 16000 records, up to 2019-05-20 20:18:29.976000
done 17233 cycles, 16500 records, up to 2019-05-20 20:20:04.275000
done 17234 cycles, 17000 records, up to 2019-05-20 20:20:46.970000
done 17235 cycles, 17500 records, up to 2019-05-20 20:21:53.851000
done 17236 cycles, 18000 records, up to 2019-05-20 20:23:14.555000
done 17237 cycles, 18500 records, up to 2019-05-20 20:24:59.807000
done 17238 cycles, 19000 records, up to 2019-05-20 20:26:29.792000
done 17239 cycles, 19500 records, up to 2019-05-20 20:27:58.934000
done 17240 cycles, 20000 records, up to 2019-05-20 20:29:58.98

done 17349 cycles, 24500 records, up to 2019-05-20 22:42:52.154000
done 17350 cycles, 25000 records, up to 2019-05-20 22:43:50.106000
done 17351 cycles, 25500 records, up to 2019-05-20 22:45:08.757000
done 17352 cycles, 26000 records, up to 2019-05-20 22:45:28.034000
done 17353 cycles, 26500 records, up to 2019-05-20 22:45:41.246000
done 17354 cycles, 27000 records, up to 2019-05-20 22:46:00.344000
done 17355 cycles, 27500 records, up to 2019-05-20 22:46:21.624000
done 17356 cycles, 28000 records, up to 2019-05-20 22:46:41.040000
done 17357 cycles, 28500 records, up to 2019-05-20 22:47:03.026000
done 17358 cycles, 29000 records, up to 2019-05-20 22:47:31.472000
done 17359 cycles, 29500 records, up to 2019-05-20 22:47:55.273000
done 17360 cycles, 30000 records, up to 2019-05-20 22:48:26.724000
done 17361 cycles, 30500 records, up to 2019-05-20 22:49:24.356000
done 17362 cycles, 31000 records, up to 2019-05-20 22:50:10.450000
done 17363 cycles, 31500 records, up to 2019-05-20 22:51:12.35

done 17472 cycles, 36000 records, up to 2019-05-21 00:36:56.757000
done 17473 cycles, 36500 records, up to 2019-05-21 00:38:10.066000
done 17474 cycles, 37000 records, up to 2019-05-21 00:39:20.536000
done 17475 cycles, 37500 records, up to 2019-05-21 00:40:31.349000
done 17476 cycles, 38000 records, up to 2019-05-21 00:41:05.676000
done 17477 cycles, 38500 records, up to 2019-05-21 00:41:41.788000
done 17478 cycles, 39000 records, up to 2019-05-21 00:42:47.112000
done 17479 cycles, 39500 records, up to 2019-05-21 00:43:51.076000
done 17480 cycles, 40000 records, up to 2019-05-21 00:44:14.705000
done 17481 cycles, 40500 records, up to 2019-05-21 00:44:58.088000
done 17482 cycles, 41000 records, up to 2019-05-21 00:45:57.377000
done 17483 cycles, 41500 records, up to 2019-05-21 00:47:11.197000
done 17484 cycles, 42000 records, up to 2019-05-21 00:48:17.325000
done 17485 cycles, 42500 records, up to 2019-05-21 00:48:40.910000
done 17486 cycles, 43000 records, up to 2019-05-21 00:49:08.09

done 17595 cycles, 47500 records, up to 2019-05-21 03:14:53.739000
done 17596 cycles, 48000 records, up to 2019-05-21 03:17:16.572000
done 17597 cycles, 48500 records, up to 2019-05-21 03:18:51.115000
done 17598 cycles, 49000 records, up to 2019-05-21 03:20:32.458000
done 17599 cycles, 49500 records, up to 2019-05-21 03:22:51.871000
done 17600 cycles, 50000 records, up to 2019-05-21 03:24:57.764000
done 17601 cycles, 500 records, up to 2019-05-21 03:26:29.755000
done 17602 cycles, 1000 records, up to 2019-05-21 03:27:26.392000
done 17603 cycles, 1500 records, up to 2019-05-21 03:28:32.140000
done 17604 cycles, 2000 records, up to 2019-05-21 03:29:54.484000
done 17605 cycles, 2500 records, up to 2019-05-21 03:31:46.655000
done 17606 cycles, 3000 records, up to 2019-05-21 03:33:35.704000
done 17607 cycles, 3500 records, up to 2019-05-21 03:35:27.440000
done 17608 cycles, 4000 records, up to 2019-05-21 03:36:44.108000
done 17609 cycles, 4500 records, up to 2019-05-21 03:38:16.927000
done 

...

done 28173 cycles, 36500 records, up to 2019-06-03 19:07:43.502000
done 28174 cycles, 37000 records, up to 2019-06-03 19:09:44.046000
done 28175 cycles, 37500 records, up to 2019-06-03 19:11:50.351000
done 28176 cycles, 38000 records, up to 2019-06-03 19:14:15.527000
done 28177 cycles, 38500 records, up to 2019-06-03 19:17:02.245000
done 28178 cycles, 39000 records, up to 2019-06-03 19:18:18.463000
done 28179 cycles, 39500 records, up to 2019-06-03 19:19:18.265000
done 28180 cycles, 40000 records, up to 2019-06-03 19:20:34.617000
done 28181 cycles, 40500 records, up to 2019-06-03 19:22:17.915000
done 28182 cycles, 41000 records, up to 2019-06-03 19:24:57.935000
done 28183 cycles, 41500 records, up to 2019-06-03 19:27:53.942000
done 28184 cycles, 42000 records, up to 2019-06-03 19:31:01.416000
done 28185 cycles, 42500 records, up to 2019-06-03 19:34:02.069000
done 28186 cycles, 43000 records, up to 2019-06-03 19:37:40.140000
done 28187 cycles, 43500 records, up to 2019-06-03 19:40:23.84

done 28296 cycles, 48000 records, up to 2019-06-04 00:11:03.284000
done 28297 cycles, 48500 records, up to 2019-06-04 00:13:59.035000
done 28298 cycles, 49000 records, up to 2019-06-04 00:16:30.692000
done 28299 cycles, 49500 records, up to 2019-06-04 00:17:42.106000
done 28300 cycles, 50000 records, up to 2019-06-04 00:18:19.181000
done 28301 cycles, 500 records, up to 2019-06-04 00:19:24.028000
done 28302 cycles, 1000 records, up to 2019-06-04 00:19:58.716000
done 28303 cycles, 1500 records, up to 2019-06-04 00:21:48.130000
done 28304 cycles, 2000 records, up to 2019-06-04 00:23:12.861000
done 28305 cycles, 2500 records, up to 2019-06-04 00:24:56.336000
done 28306 cycles, 3000 records, up to 2019-06-04 00:27:26.230000
done 28307 cycles, 3500 records, up to 2019-06-04 00:29:07.348000
done 28308 cycles, 4000 records, up to 2019-06-04 00:30:37.331000
done 28309 cycles, 4500 records, up to 2019-06-04 00:32:34.343000
done 28310 cycles, 5000 records, up to 2019-06-04 00:34:19.987000
done 2

done 28419 cycles, 9500 records, up to 2019-06-04 02:01:01.712000
done 28420 cycles, 10000 records, up to 2019-06-04 02:01:10.234000
done 28421 cycles, 10500 records, up to 2019-06-04 02:01:20.409000
done 28422 cycles, 11000 records, up to 2019-06-04 02:01:32.399000
done 28423 cycles, 11500 records, up to 2019-06-04 02:01:51.335000
done 28424 cycles, 12000 records, up to 2019-06-04 02:02:11.962000
done 28425 cycles, 12500 records, up to 2019-06-04 02:02:32.230000
done 28426 cycles, 13000 records, up to 2019-06-04 02:02:54.768000
done 28427 cycles, 13500 records, up to 2019-06-04 02:03:17.731000
done 28428 cycles, 14000 records, up to 2019-06-04 02:03:35.605000
done 28429 cycles, 14500 records, up to 2019-06-04 02:03:50.540000
done 28430 cycles, 15000 records, up to 2019-06-04 02:04:10.367000
done 28431 cycles, 15500 records, up to 2019-06-04 02:04:28.963000
done 28432 cycles, 16000 records, up to 2019-06-04 02:04:50.519000
done 28433 cycles, 16500 records, up to 2019-06-04 02:04:59.996

done 28542 cycles, 21000 records, up to 2019-06-04 03:09:43.318000
done 28543 cycles, 21500 records, up to 2019-06-04 03:10:34.766000
done 28544 cycles, 22000 records, up to 2019-06-04 03:12:01.117000
done 28545 cycles, 22500 records, up to 2019-06-04 03:13:12.672000
done 28546 cycles, 23000 records, up to 2019-06-04 03:14:07.513000
done 28547 cycles, 23500 records, up to 2019-06-04 03:14:49.013000
done 28548 cycles, 24000 records, up to 2019-06-04 03:15:43.117000
done 28549 cycles, 24500 records, up to 2019-06-04 03:16:50.736000
done 28550 cycles, 25000 records, up to 2019-06-04 03:18:13.846000
done 28551 cycles, 25500 records, up to 2019-06-04 03:19:30.188000
done 28552 cycles, 26000 records, up to 2019-06-04 03:20:42.977000
done 28553 cycles, 26500 records, up to 2019-06-04 03:21:56.741000
done 28554 cycles, 27000 records, up to 2019-06-04 03:23:22.972000
done 28555 cycles, 27500 records, up to 2019-06-04 03:24:37.609000
done 28556 cycles, 28000 records, up to 2019-06-04 03:26:10.44

NameError: name 'sys' is not defined

In [147]:
data

,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,openTime,close,closeTime,high,low,numTrades,open,...,is_h100,ma20,ma100,ma500,cons_mvt,tot_mvt,ma500_trend,buyer_is_maker,profit,immo_time
99,99,99,99,2018-12-16 05:00:00,3278.04,2018-12-16 05:00:59.999,3293.64,3283.82,2393,3273.58,...,1,3239.1200,3215.6381,3215.638100,1.0,4.46,1,0.553220,NaN,NaN
100,100,100,100,2018-12-16 05:15:00,3285.22,2018-12-16 05:15:59.999,3286.37,3284.84,1334,3277.72,...,1,3242.5895,3216.0200,3216.327030,2.0,11.96,1,0.611154,NaN,NaN
101,101,101,101,2018-12-16 05:30:00,3273.98,2018-12-16 05:30:59.999,3285.25,3282.04,1367,3285.00,...,1,3245.5655,3216.2883,3216.892255,2.0,-22.04,1,0.443155,NaN,NaN
102,102,102,102,2018-12-16 05:45:00,3264.00,2018-12-16 05:45:59.999,3274.00,3270.00,1400,3273.25,...,0,3248.1795,3216.3661,3217.349612,3.0,-31.29,1,0.356123,NaN,NaN
103,103,103,103,2018-12-16 06:00:00,3269.96,2018-12-16 06:00:59.999,3271.61,3269.08,1152,3263.99,...,0,3250.9260,3216.5757,3217.855481,1.0,5.97,1,0.382805,NaN,NaN
104,104,104,104,2018-12-16 06:15:00,3266.11,2018-12-16 06:15:59.999,3270.66,3269.26,873,3269.53,...,0,3253.1250,3216.6582,3218.315048,2.0,-6.84,1,0.435503,NaN,NaN
105,105,105,105,2018-12-16 06:30:00,3270.94,2018-12-16 06:30:59.999,3271.53,3268.05,900,3265.67,...,0,3255.8890,3216.7345,3218.811509,1.0,5.27,1,0.479862,NaN,NaN
106,106,106,106,2018-12-16 06:45:00,3269.39,2018-12-16 06:45:59.999,3273.77,3271.26,834,3270.00,...,0,3258.6585,3216.8422,3219.284206,2.0,-1.22,1,0.597671,NaN,NaN
107,107,107,107,2018-12-16 07:00:00,3278.50,2018-12-16 07:00:59.999,3278.93,3276.00,1025,3270.33,...,1,3261.7080,3217.1076,3219.832500,1.0,8.17,1,0.549383,NaN,NaN
108,108,108,108,2018-12-16 07:15:00,3281.98,2018-12-16 07:15:59.999,3282.00,3279.03,1259,3276.61,...,1,3264.7635,3217.4816,3220.402661,2.0,13.54,1,0.637705,NaN,NaN
